In [16]:
import os
import numpy as np
from PIL import Image

In [17]:
def export_png_from_ROIlist(ROIfile_withpath, outputpath, ROInumbers=None):
    # Get the base directory, filename, and extension
    basedir, filename_ext = os.path.split(ROIfile_withpath)
    filename, ext = os.path.splitext(filename_ext)
    
    # Create the output path if it doesn't exist
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
    
    # Load ADC data with correct delimiter
    adcfile = os.path.join(basedir, filename + '.adc')
    adcdata = np.loadtxt(adcfile, delimiter=',')
    
    # Determine column indices based on filename format
    if filename.startswith('I'):
        x = adcdata[:, 11]
        y = adcdata[:, 12]
        startbyte = adcdata[:, 13]
    else:
        x = adcdata[:, 15]
        y = adcdata[:, 16]
        startbyte = adcdata[:, 17]
    
    # If no ROInumbers are passed, export all
    if ROInumbers is None:
        ROInumbers = np.where(x > 0)[0] + 1  # MATLAB indices start from 1, Python from 0
    
    roi_filepath = ROIfile_withpath + '.roi'
    with open(roi_filepath, 'rb') as fid:
        for num in ROInumbers:
            num = int(num) - 1  # Adjusting MATLAB 1-based index to Python 0-based index
            fid.seek(int(startbyte[num]), os.SEEK_SET)
            img_data = np.fromfile(fid, dtype=np.uint8, count=int(x[num] * y[num]))
            if img_data.size != int(x[num] * y[num]):
                print(f"Warning: ROI {num + 1} has unexpected size. Skipping.")
                continue
            img = np.reshape(img_data, (int(y[num]), int(x[num])))  # Note the order of x and y for correct orientation
            pngname = f"{filename}_{num + 1:05d}.png"
            img = Image.fromarray(img)
            img.save(os.path.join(outputpath, pngname))

def process_all_roi_files(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.roi'):
                roi_file_path = os.path.join(root, file)
                filename = os.path.splitext(file)[0]
                output_path = os.path.join(output_dir, filename)  # Create a subdirectory for each .roi file
                export_png_from_ROIlist(roi_file_path[:-4], output_path)  # Remove the '.roi' extension



In [18]:
# Example usage
input_directory = 'C:/Data/Python/IFCB/data/2024/raw'
output_directory = 'C:/Data/Python/IFCB/data/2024/images'
process_all_roi_files(input_directory, output_directory)